In [1]:
from os import walk
import pandas as pd
import numpy as np
from PIL import Image
from sklearn import preprocessing

# Classification part

In [2]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1/255.)
valid_datagen = ImageDataGenerator(rescale=1/255.)

In [3]:
train_generator = train_datagen.flow_from_directory('train', 
                        target_size=(250, 250), batch_size=20)
valid_generator = valid_datagen.flow_from_directory('validation',
                        target_size=(250, 250), batch_size=20)

Found 3068 images belonging to 10 classes.
Found 341 images belonging to 10 classes.


In [4]:
#from keras.applications.vgg16 import VGG16
from keras.applications import VGG19
from keras.layers import Flatten, Dense, Input, Dropout
from keras.models import Model
from keras.optimizers import Adam

vgg = VGG19(weights="imagenet", include_top=False, 
            input_tensor=Input(shape=(250, 250, 3)))
vgg.trainable = False


flatten = vgg.output
flatten = Flatten()(flatten)
bboxHead = Dense(128, activation="relu")(flatten)
#bboxHead = Dropout(.3)(bboxHead)
bboxHead = Dense(64, activation="relu")(bboxHead)
#bboxHead = Dropout(.2)(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
#bboxHead = Dropout(.1)(bboxHead)
bboxHead = Dense(10, activation="softmax")(bboxHead)
# construct the model we will fine-tune for bounding box regression
model = Model(inputs=vgg.input, outputs=bboxHead)

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 250, 250, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 250, 250, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 250, 250, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 125, 125, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 125, 125, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 125, 125, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 62, 62, 128)       0     

In [6]:
#LR = 1e-4
opt = Adam()
model.compile(loss="categorical_crossentropy", 
              optimizer=opt,  
              metrics=['acc'])

In [7]:
model.compile(loss='categorical_crossentropy', metrics=['acc'])
model.fit_generator(train_generator, epochs=10, 
                             validation_data=valid_generator)

C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
154/154 [==============================] - 25s 126ms/step - loss: 2.3101 - acc: 0.3247 - val_loss: 0.8497 - val_acc: 0.7273
Epoch 2/10
154/154 [==============================] - 17s 110ms/step - loss: 0.8220 - acc: 0.7298 - val_loss: 0.6148 - val_acc: 0.7889
Epoch 3/10
154/154 [==============================] - 17s 110ms/step - loss: 0.5252 - acc: 0.8193 - val_loss: 0.5928 - val_acc: 0.8065
Epoch 4/10
154/154 [==============================] - 17s 111ms/step - loss: 0.3441 - acc: 0.8748 - val_loss: 0.6896 - val_acc: 0.7889
Epoch 5/10
154/154 [==============================] - 17s 111ms/step - loss: 0.2524 - acc: 0.9096 - val_loss: 0.7824 - val_acc: 0.7771
Epoch 6/10
154/154 [==============================] - 17s 111ms/step - loss: 0.1975 - acc: 0.9347 - val_loss: 0.7098 - val_acc: 0.8035
Epoch 7/10
154/154 [==============================] - 17s 111ms/step - loss: 0.1515 - acc: 0.9539 - val_loss: 0.4563 - val_acc: 0.8622
Epoch 8/10
154/154 [==============================] - 1

In [8]:
model.save("classificator_.hd5")

INFO:tensorflow:Assets written to: classificator_.hd5\assets


In [9]:
label_map = (train_generator.class_indices)

In [10]:
label_map

{'dress': 0,
 'hat': 1,
 'longsleeve': 2,
 'outwear': 3,
 'pants': 4,
 'shirt': 5,
 'shoes': 6,
 'shorts': 7,
 'skirt': 8,
 't-shirt': 9}